In [1]:
import cv2
import numpy as np
import os
import glob

IMG_SIZE = 512  # 设置图像大小为 512x512

def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:  # 图像太暗，将裁剪所有内容，返回原始图像
            return img
        else:
            img1 = img[:,:,0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:,:,1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:,:,2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img

def circle_crop(img, sigmaX):
    """
    在图像中心创建圆形裁剪
    """
    img = crop_image_from_gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    height, width, depth = img.shape
    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), sigmaX), -4, 128)
    
    return img

In [2]:
import cv2
import numpy as np
import os
import glob

IMG_SIZE = 512  # 设置图像大小为 512x512

# 定义 crop_image_from_gray 和 circle_crop 函数

source_folders = [
    '/kaggle/input/new-mirrored-d-n-raw/D_N_train_images/D_N_train_images',
    '/kaggle/input/new-mirrored-d-n-raw/D_N_validation_images/D_N_validation_images',
    '/kaggle/input/new-mirrored-d-n-raw/D_N_test_images/D_N_test_images'
]

target_folders = [
    '/kaggle/working/gaus_train',
    '/kaggle/working/gaus_validation',
    '/kaggle/working/gaus_test'
]

for source_folder, target_folder in zip(source_folders, target_folders):
    # 确保目标文件夹存在
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # 获取源文件夹中所有图像的文件路径
    image_paths = glob.glob(os.path.join(source_folder, '*.jpg'))  # 假设图片为.jpg格式，根据需要调整
    
    for image_path in image_paths:
        # 读取和处理图像
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # 确保图像是512x512
        processed_img = circle_crop(img, sigmaX=30)
        
        # 构建目标文件路径
        target_image_path = os.path.join(target_folder, os.path.basename(image_path))
        
        # 保存处理后的图像
        cv2.imwrite(target_image_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))

print("Image processing and saving completed.")


Image processing and saving completed.
